In [1]:
import sys
import os
import numpy as np
import urllib

In [2]:
class PDBEntry:
    def __init__(self, idx, name, chain):
        self.indexNum = idx
        self.name = name
        self.chain = chain

In [3]:
class PCassoAnalysisAA:
    def __init__(self, chain, AAName, SSType, location):
        self.chain = chain
        self.SSType = SSType
        self.AAName = AAName
        self.location = location

In [4]:
class PCassoAnaylsisPDB:
    def __init__ (self, PDBName):
        self.PDBName = PDBName
        self.AAs = []
    

In [5]:
class PDBPDB:
    def __init__(self, name):
        self.name = name
        self.minSeqNum = 0
        self.maxSeqNum = 0
        self.AAs = []
        self.helices = []
        self.sheets = []
        self.predictions = []
        self.helixSeqNums = []
        self.sheetSeqNums = []
        
    def calcHelixSeqNums(self, chain):
        for h in self.helices:
            if(h.chainID == chain):
                start = h.initSeqNum
                end = h.endSeqNum + 1
                for x in range(start,end):
                    self.helixSeqNums.append(x)

    def calcSheetSeqNums(self, chain):
        for s in self.sheets:
            if(s.startChain == chain):
                start = int(s.initSeqNum)
                end = int(s.endSeqNum) + 1
                for x in range(start,end):
                    if x not in self.sheetSeqNums:
                        self.sheetSeqNums.append(x)
        if(self.name == "3SSB"):        
            print(f'found sheet in PDB {self.name}, seqnums {self.sheetSeqNums}')
                               
    def checkSeqNum(self,seqNum):
        if seqNum in self.helixSeqNums:
            return 'H'
        if seqNum in self.sheetSeqNums:
            return 'S'
        else:
            return 'L'

In [6]:
class Helix:
    def __init__ (self, serNum, helixID, firstRes, chainID, initSeqNum, lastRes, endChainID, endSeqNum):
        self.serNum = serNum
        self.helixID = helixID
        self.firstRes = firstRes
        self.chainID = chainID
        self.initSeqNum = initSeqNum
        self.lastRes = lastRes
        self.endChainID = endChainID
        self.endSeqNum = endSeqNum

In [7]:
class Sheet:
    def __init__ (self, strand, sheetID, startChain, initSeqNum, endChain, endSeqNum):
        self.strand = strand
        self.sheetID = sheetID
        self.startChain = startChain
        self.initSeqNum = initSeqNum
        self.endChain = endChain
        self.endSeqNum = endSeqNum
#         print(f'In Sheet class startChain is {startChain}')

In [8]:
class Prediction:
    def __init__ (self, name, chain, seqNum, PDBType, PCASSOType):
        self.name = name
        self.chain = chain
        self.seqNum = seqNum
        self.PDBType = PDBType
        self.PCASSOType = PCASSOType

In [9]:
def download_pdb(pdbcode, datadir, downloadurl="https://files.rcsb.org/download/"):
    """
    Downloads a PDB file from the Internet and saves it in a data directory.
    :param pdbcode: The standard PDB ID e.g. '3ICB' or '3icb'
    :param datadir: The directory where the downloaded file will be saved
    :param downloadurl: The base PDB download URL, cf.
        `https://www.rcsb.org/pages/download/http#structures` for details
    :return: the full path to the downloaded PDB file or None if something went wrong
    """
    pdbfn = pdbcode + ".pdb"
    url = downloadurl + pdbfn
    outfnm = os.path.join(datadir, pdbfn)
    try:
        urllib.request.urlretrieve(url, outfnm)
        print(outfnm)
        return outfnm
    except Exception as err:
        print("ERROR!!")
        print(str(err), file=sys.stderr)
        return None


In [10]:
## This is the original content of this cell, used for reading from .csv file to get list of PDBs to examine
## Removed to allow access to single PDBs
notebook_path = os.path.abspath("Notebook.ipynb")
# PDBList = os.path.join(os.path.dirname(notebook_path), "DataSet/Set_for_table_X.csv")
PDBList = os.path.join(os.path.dirname(notebook_path), "DataSet/Test_Set_300.csv")
PDBs = []
with open(PDBList) as file:
    for item in file:
        item = item.rstrip('\n')
        tmp = item.split(',')
        obj = PDBEntry(tmp[0], tmp[1],tmp[2])
        PDBs.append(obj)
print(len(PDBs))

300


In [11]:
# notebook_path = os.path.abspath("Notebook.ipynb")
# PDBs = []
# # item = "1,3SSB,C"
# # item = "2,2END,A"
# # item = "3,1ZUU,A"
# # item = "4,4UE8,A"
# # item = "5,5W82,A"
# # item = "6,3QR7,A"
# # item = "7,3NGG,A"
# # item = "8,3X34,A"
# # item = "9,1KVE,A"
# # item = "10,5DBL,A"
# # item = "11,4KK7,A"
# # item = "12,3MAO,A"
# # item = "13,5QS9,A"
# # item = "14,6DWD,D"
# # item = "15,4G9S,A"
# # item = "16,3ZVS,A"
# # item = "17,3QL9,A"
# # item = "18,4ZFL,A"
# # item = "19,5OBY,A"
# # item = "20,6B1K,A"
# # item = "21,1T7M,B"
# # item = "22,2IC6,A"
# # item = "23,3HE5,B"
# # item = "24,3LDC,A"
# # item = "25,4ABM,A"
# # item = "26,4I6R,A"
# # item = "27,4WZX,A"
# # item = "28,5O17,A"
# # item = "29,5TZQ,C"
# item = "30,6OP8,B"
# item = item.rstrip('\n')
# tmp = item.split(',')
# obj = PDBEntry(tmp[0], tmp[1],tmp[2])
# PDBs.append(obj)

# The next cell downloads the pdb files from the PDB database site.

You only need to run it once.

In [12]:
for pdb in PDBs:
    
    path = os.path.join(os.path.dirname(notebook_path),"PDBFiles",pdb.name )
    if(not os.path.isdir(path)):
        print(path)
        os.mkdir(path)

    download_pdb(pdb.name, path)

/home/chris/projects/PCASSOPaper/PDBFiles/1C0P/1C0P.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1DG6/1DG6.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1GCI/1GCI.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1GS9/1GS9.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1HZT/1HZT.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1I4J/1I4J.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1INL/1INL.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1KVE/1KVE.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1LLF/1LLF.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1M22/1M22.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1NKD/1NKD.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1O06/1O06.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1O98/1O98.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1PO5/1PO5.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1QW9/1QW9.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1R0U/1R0U.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1R6X/1R6X.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/1R9L/1

/home/chris/projects/PCASSOPaper/PDBFiles/4D0P/4D0P.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4DMV/4DMV.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4EMN/4EMN.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4EO0/4EO0.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4ERN/4ERN.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4G9S/4G9S.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4GT8/4GT8.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4HHJ/4HHJ.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4I6R/4I6R.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4I6Y/4I6Y.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4I79/4I79.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4I84/4I84.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4JCC/4JCC.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4JIF/4JIF.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4K5A/4K5A.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4K8W/4K8W.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4KK7/4KK7.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/4KT6/4

/home/chris/projects/PCASSOPaper/PDBFiles/6RNI/6RNI.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/6SSH/6SSH.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/6SYV/6SYV.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/6T02/6T02.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/6T0Y/6T0Y.pdb
/home/chris/projects/PCASSOPaper/PDBFiles/6UQV/6UQV.pdb


In [13]:
for pdb in PDBs:    
    path = os.path.join(os.path.dirname(notebook_path),"PDBFiles",pdb.name )
    command = "/home/chris/PCASSO/bin/pcasso -verbose " + path + "/" + pdb.name + ".pdb > "
    command = command + "/home/chris/projects/PCASSOPaper/PDBFiles/" + pdb.name +"/" + pdb.name + ".pcasso"
    os.system(command)

Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1C0P/1C0P.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1DG6/1DG6.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1GCI/1GCI.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1GS9/1GS9.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1HZT/1HZT.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1I4J/1I4J.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1INL/1INL.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1KVE/1KVE.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1LLF/1LLF.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1M22/1M22.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1NKD/1NKD.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1O06/1O06.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/1O98/1O98.pdb...
Processing f

Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3NGG/3NGG.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3NRS/3NRS.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3PE6/3PE6.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3PIW/3PIW.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3Q1C/3Q1C.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3QL9/3QL9.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3QR7/3QR7.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3QY7/3QY7.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3RET/3RET.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3RO3/3RO3.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3S0A/3S0A.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3S2R/3S2R.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/3S8G/3S8G.pdb...
Processing f

Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5IHW/5IHW.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5J1N/5J1N.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5JAZ/5JAZ.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5JE2/5JE2.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5JO8/5JO8.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5JVI/5JVI.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5KO5/5KO5.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5LE5/5LE5.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5LE5/5LE5.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5LJX/5LJX.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5LW3/5LW3.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5M10/5M10.pdb...
Processing file "/home/chris/projects/PCASSOPaper/PDBFiles/5M33/5M33.pdb...
Processing f

# All files should be downloaded and analysis by PCASSO

In [14]:
for pdb in PDBs:
    tempPCPDB = PCassoAnaylsisPDB(pdb.name) 
    path = os.path.join(os.path.dirname(notebook_path),"PDBFiles",pdb.name)
    path = path + "/" + pdb.name + ".pcasso" 

    file1 = open(path, 'r')
    temp = file1.readline()
    changed = False
    if(temp.find(' :') >= 0):
        temp = temp.replace(" :", "zz:")
        changed = True
    
    file1.close()
    if(changed):
        file1 = open(path, 'w')
        file1.write(temp)
        file1.close()    

In [15]:
allPCassoPDBs = []
for pdb in PDBs:
    tempPCPDB = PCassoAnaylsisPDB(pdb.name) 
    path = os.path.join(os.path.dirname(notebook_path),"PDBFiles",pdb.name)
    path = path + "/" + pdb.name + ".pcasso" 

    file1 = open(path, 'r')
    temp = file1.readline()

    temp = temp.strip()
    tempArr = temp.split(' ')

    for x in range(2, len(tempArr), 2):
        tempArr[x] = tempArr[x].strip()

        chainAndName = tempArr[x].split(':')
        if(chainAndName[0] != 'zz'):
            chain = chainAndName[0]

            AAName = chainAndName[1][0:3]
            AAlocation = chainAndName[1][3:len(chainAndName[1]) -3]
            SSType = tempArr[x+1]
        else:
            chain = ""
            AAName = chainAndName[0][0:3]
            AAlocation = chainAndName[0][3:len(chainAndName[0]) -3]
            SSType = tempArr[x+1]

        tempAA = PCassoAnalysisAA(chain, AAName, SSType, AAlocation)
        tempPCPDB.AAs.append(tempAA)
        
    
    allPCassoPDBs.append(tempPCPDB)
    file1.close()
print(len(allPCassoPDBs))
sCount = 0
for pdb in allPCassoPDBs:
    for aa in pdb.AAs:
#         print(aa.chain, aa.AAName, aa.SSType)
        if aa.SSType == "E":
            sCount = sCount + 1
print(f'Sheet Count {sCount}')

300
Sheet Count 26771


In [16]:
temp = 0
allProteinDataBankPDBs = []
for pdb in PDBs:
    tempPDBPDB = PDBPDB(pdb.name) 
    path = os.path.join(os.path.dirname(notebook_path),"PDBFiles",pdb.name)
    path = path + "/" + pdb.name + ".pdb" 

    file1 = open(path, 'r')
    Lines = file1.readlines()
 

    minSeqNum = sys.maxsize
    maxSeqNum = -sys.maxsize - 1

    for line in Lines:
        if(line[:6] == "ATOM  "):
            thisChain = line[21]
            if(thisChain == pdb.chain):
                thisSeqNum = int(line[23:26])
#                 print(thisSeqNum)
                if(thisSeqNum < minSeqNum):
                    minSeqNum = thisSeqNum
                if(thisSeqNum > maxSeqNum):
                     maxSeqNum = thisSeqNum
                    
        if(line[:5] == "HELIX"):
                serNum = int(line[7:10])
                helixID = line[12:14]
                firstRes = line[15:18]
                chainID = line[19]
                initSeqNum = int(line[21:25])
                lastRes = line[27:30]
                endChainID = line[31]
                endSeqNum = int(line[33:37])
                tempHelix = Helix(serNum, helixID, firstRes, chainID, initSeqNum, lastRes, endChainID,endSeqNum)
                tempPDBPDB.helices.append(tempHelix)
        if(line[:5] == "SHEET"): 
                strand = int(line[8:10])
                sheetID = line[12:14]
                startChain = line[21]

                initSeqNum = line[22:26]
                endChain = line[32]
                endSeqNum = line[33:37]
                tempSheet = Sheet(strand , sheetID, startChain, initSeqNum, endChain, endSeqNum)
#                 if(pdb.name == "3SSB"):
#                     print(startChain, tempSheet.startChain)
                tempPDBPDB.sheets.append(tempSheet)
                temp = temp + 1
    tempPDBPDB.minSeqNum = minSeqNum
    tempPDBPDB.maxSeqNum = maxSeqNum
    allProteinDataBankPDBs.append(tempPDBPDB)
    file1.close()
# for pdb in allProteinDataBankPDBs:
#     if(pdb.name == "3SSB"):
#         for s in pdb.sheets:
#             print(f'SheetID {s.sheetID}, strand {s.strand}, startChain: {s.startChain}, endChain: {s.endChain}, startSeq {s.initSeqNum}, endSeq {s.endSeqNum}')
# #                     print(f'sheet {temp}:: {tempSheet.initSeqNum} to {tempSheet.endSeqNum}')
sCount = 0
for pdb in allProteinDataBankPDBs:
    for sheet in pdb.sheets:
        sCount = sCount + 1
print(f'Sheet Count {sCount}')

Sheet Count 5542


In [17]:
# allProteinDataBankPDBs
len(allProteinDataBankPDBs)

300

In [18]:
# class PCassoAnalysisAA:
#     def __init__(self, chain, AAName, SSType, location):
#         self.chain = chain
#         self.SSType = SSType
#         self.AAName = AAName
#         self.location = location
# count = 0
# for PCPDB in allPCassoPDBs:
#     if PCPDB.PDBName == "1T7M":
#         for aa in PCPDB.AAs:
#             if(aa.chain == "B"):
#                 print(f'aa.chain {aa.chain}, seqNum {aa.location}, count {count}')
#     count = count + 1

In [19]:
count = 0
predictions = []
for pdb in PDBs:
#     print(f'Processing {pdb.name}')
#     print(allProteinDataBankPDBs[count], allPCassoPDBs[count])
#     print(pdb.chain)
#     print(allProteinDataBankPDBs[count].helices)
    PDBPredict = allProteinDataBankPDBs[count]
    start = PDBPredict.minSeqNum
    end = PDBPredict.maxSeqNum + 1
    PCASSOPredict = allPCassoPDBs[count]
    PDBPredict.calcHelixSeqNums(pdb.chain)
    PDBPredict.calcSheetSeqNums(pdb.chain)
    for aa in PCASSOPredict.AAs:
        if(aa.chain == pdb.chain):
            for seqNum in range(start,end):
                if(int(aa.location) == int(seqNum)):
                    PCASSOType = aa.SSType
                    PDBType = PDBPredict.checkSeqNum(int(aa.location))
                    tempPredict = Prediction(PDBPredict.name, aa.chain, seqNum, PDBType, PCASSOType)
                    predictions.append(tempPredict)

    count = count + 1
# count = 0
# for pdb in PDBs:
#     PDBPredict = allProteinDataBankPDBs[count]
#     if(PDBPredict.name == "3SSB"):
# #         print(PDBPredict.minSeqNum, PDBPredict.maxSeqNum)
#         for seq in range(PDBPredict.minSeqNum, (PDBPredict.maxSeqNum + 1)):
#             print(seq, PDBPredict.checkSeqNum(int(seq)))
#     count = count + 1

                        
# for predict in predictions:
#     if(predict.PDBType == "S"):
#         print(predict.PDBType, predict.PCASSOType)
#     print("listing")
#     for prediction in predictions:
#         if(prediction.PDBType != prediction.PCASSOType):
#             print(f'Protein ID: {prediction.name}, Chain: {prediction.chain}, Seq Num: {prediction.seqNum}, PDB Type: {prediction.PDBType}, PCASSO Type {prediction.PCASSOType}')
    
# class Prediction:
#     def __init__ (self, name, chain, seqNum, PDBType, PCASSOType):
#         self.name = name
#         self.chain = chain
#         self.seqNum = seqNum
#         self.PDBType = PDBType
#         self.PCASSOType = PCASSOType
    

#     for seqNum in range(start,end):
#         if(pdb.chain == PCASSOPredict.AAs[seqNum].chain):
#             PCASSOType = PCASSOPredict.AAs[seqNum].SSType
#             PDBType = 'C'
#             if(seqNum in helixSeqNums):
#                 PDBType = "H"
#             if(seqNum in sheetSeqNums):
#                 PDBType = "E"
#             tempPredict = Prediction(seqNum, PDBType, PCASSOPredict.AAs[seqNum].SSType)
# #             print(tempPredict.seqNum, tempPredict.PDBType, tempPredict.PCASSOType)
        
    
    

found sheet in PDB 3SSB, seqnums [60, 61, 62, 63, 64, 65, 69, 70, 71, 77, 78, 79]


In [20]:
    tP_Helix = 0;
    tP_Sheet = 0;
    tP_Loop = 0;
    
    fp_HelixSheet = 0;
    fp_HelixLoop = 0;

    fp_SheetHelix = 0;
    fp_SheetLoop = 0;
    
    fp_LoopHelix = 0;
    fp_LoopSheet = 0;
    
    for prediction in predictions:
#         print(prediction.PDBType)
        if(prediction.PDBType == 'H'):
            if(prediction.PCASSOType == 'H'):
                tP_Helix = tP_Helix + 1
            elif(prediction.PCASSOType == 'E'):
                fp_HelixSheet = fp_HelixSheet + 1
            else:
                fp_HelixLoop = fp_HelixLoop + 1
#         print(prediction.PDBType)
        if(prediction.PDBType == 'S'):
#             print(prediction.PDBType)
            if(prediction.PCASSOType == 'E'):
                tP_Sheet = tP_Sheet + 1
            elif(prediction.PCASSOType == 'H'):
                fp_SheetHelix = fp_SheetHelix + 1
            else:
                fp_SheetLoop = fp_SheetLoop + 1
                
        if(prediction.PDBType == 'L'):
            if(prediction.PCASSOType == 'C'):
                tP_Loop = tP_Loop + 1
            elif(prediction.PCASSOType == 'H'):
                fp_LoopHelix = fp_LoopHelix + 1
            else:
                fp_LoopSheet = fp_LoopSheet + 1
#         if(prediction.PDBType != prediction.PCASSOType):
#             print(f'Protein ID: {prediction.name}, Chain: {prediction.chain}, Seq Num: {prediction.seqNum}, PDB Type: {prediction.PDBType}, PCASSO Type {prediction.PCASSOType}')
    print("---====== HELIX ======---")
    print(f'    True Helix: {tP_Helix}')
    print(f'    FP Sheet {fp_HelixSheet}')
    print(f'    FP loop {fp_HelixLoop}')
    print()
    helix_Total = tP_Helix + fp_HelixSheet + fp_HelixLoop
    print("---====== SHEET ======---")
    print(f'    True Sheet: {tP_Sheet}')
    print(f'    FP Helix {fp_SheetHelix}')
    print(f'    FP loop {fp_SheetLoop}')
    print()
    sheet_Total = tP_Sheet + fp_SheetHelix + fp_SheetLoop
    print("---====== LOOP ======---")
    print(f'    True Loop: {tP_Loop}')
    print(f'    FP Helix {fp_LoopHelix}')
    print(f'    FP Sheet {fp_LoopSheet}')
    print()
    loop_Total = tP_Loop + fp_LoopHelix + fp_LoopSheet
    total_checked = helix_Total + sheet_Total + loop_Total
    total_correct = tP_Helix + tP_Sheet + tP_Loop
    print(f'total predictions checked = {total_checked} of {len(predictions)}, total correct {total_correct}')
    accuracy = total_correct / len(predictions) 
    print(f'Accuracy = {accuracy}')
    print(f'Percent Accuracy {accuracy*100}')

---====== HELIX ======---
    True Helix: 24077
    FP Sheet 53
    FP loop 6132

---====== SHEET ======---
    True Sheet: 12117
    FP Helix 6
    FP loop 1663

---====== LOOP ======---
    True Loop: 20249
    FP Helix 432
    FP Sheet 1188

total predictions checked = 65917 of 65917, total correct 56443
Accuracy = 0.8562737988682737
Percent Accuracy 85.62737988682737
